tennis players and grand slams

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Initialize SparkSession
spark = SparkSession.builder.appName("tennis").getOrCreate()

In [3]:
players_data = [
    (1, "Nadal"),
    (2, "Federer"),
    (3, "Novak")
]

players_columns = ["player_id","player_name"]

players_df = spark.createDataFrame(players_data,players_columns)
players_df.show()

+---------+-----------+
|player_id|player_name|
+---------+-----------+
|        1|      Nadal|
|        2|    Federer|
|        3|      Novak|
+---------+-----------+



In [16]:
championships_data = [
    (2017, 2, 1, 1, 2),
    (2018, 3, 1, 3, 2),
    (2019, 3, 1, 1, 3)
]

championships_columns= ['year','Wimbledon', 'Fr_open', 'US_open', 'Au_open']

championships_df = spark.createDataFrame(championships_data,championships_columns)

championships_df.show()


+----+---------+-------+-------+-------+
|year|Wimbledon|Fr_open|US_open|Au_open|
+----+---------+-------+-------+-------+
|2017|        2|      1|      1|      2|
|2018|        3|      1|      3|      2|
|2019|        3|      1|      1|      3|
+----+---------+-------+-------+-------+



SQL APPROACH 

In [8]:
players_df.createOrReplaceTempView("players")
championships_df.createOrReplaceTempView("championships")

In [9]:
query = """
SELECT p.player_id, p.player_name,
    SUM(
        CASE WHEN c.Wimbledon = p.player_id THEN 1 ELSE 0 END +
        CASE WHEN c.Fr_open = p.player_id THEN 1 ELSE 0 END +
        CASE WHEN c.US_open = p.player_id THEN 1 ELSE 0 END +
        CASE WHEN c.Au_open = p.player_id THEN 1 ELSE 0 END
    ) AS grand_slams_count
FROM players p
JOIN championships c ON c.Wimbledon = p.player_id OR c.Fr_open = p.player_id OR c.US_open = p.player_id OR c.Au_open = p.player_id
GROUP BY 1, 2
ORDER BY player_id DESC;
 """


In [10]:

result = spark.sql(query)

result.show()

+---------+-----------+-----------------+
|player_id|player_name|grand_slams_count|
+---------+-----------+-----------------+
|        3|      Novak|                4|
|        2|    Federer|                3|
|        1|      Nadal|                5|
+---------+-----------+-----------------+



Pyspark code continuation

In [17]:
cte1 = (championships_df.select('year',col('Wimbledon').alias('player'))\
                        .unionAll(championships_df.select('year',col('Fr_open').alias('player')))\
                        .unionAll(championships_df.select('year',col('US_open').alias('player')))\
                        .unionAll(championships_df.select('year',col('Au_open').alias('player'))))

cte1.show()                      

+----+------+
|year|player|
+----+------+
|2017|     2|
|2018|     3|
|2019|     3|
|2017|     1|
|2018|     1|
|2019|     1|
|2017|     1|
|2018|     3|
|2019|     1|
|2017|     2|
|2018|     2|
|2019|     3|
+----+------+



aggregate and join to get expected output

In [19]:
cte2 = cte1.groupBy('player').agg(count('*').alias('grand_slams_count'))

cte2.show()
                                   

+------+-----------------+
|player|grand_slams_count|
+------+-----------------+
|     2|                3|
|     3|                4|
|     1|                5|
+------+-----------------+



join operations

In [20]:
result_df = cte2.join(players_df,cte2.player==players_df.player_id,"inner")\
                .select('player_id','player_name','grand_slams_count')

result_df.show()

+---------+-----------+-----------------+
|player_id|player_name|grand_slams_count|
+---------+-----------+-----------------+
|        1|      Nadal|                5|
|        2|    Federer|                3|
|        3|      Novak|                4|
+---------+-----------+-----------------+

